# Cyclic Weight Transfer (CWT) with Cyclic Workflow

In this example, we will demonstrate the Cyclic workflow using the Client API with the CIFAR10 dataset. 

## Cyclic Workflow
<a id = "cyclic_workflow"></a>

Cyclic Weight Transfer (CWF) uses the server-controlled `CyclicController` to pass the model weights from one site to the next in a cyclic fashion. 

In the Cyclic workflow, sites train one at a time, while sending the model to the next site. The order of the sites can be specifed as fixed, random, or random (without same in a row).  A round is finished once all sites in the defined order have completed training once, and the final result is returned to the server. This differs from Scatter-and-Gather, wherein all sites train simultaneously and aggregrate their results together at the end of a round.

## Converting DL training code to FL training code

We will be using the [Client API FL code](../code/fl/train.py) trainer converted from the original [Training a Classifer](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) example.

See [Converting to FL code using Client API](../sag/sag.ipynb#code) for more details.

## Job Configuration

The client configuration for the trainer with the Client API is standard with the `PTClientAPILauncherExecutor`, `SubprocessLauncher`, and our defined `train.py` that supports the `train` task. This is the same configuration used in the SAG pt workflow.

For the server configuration we use the `CyclicController` for the workflow, and define arguments such as number of rounds, order of relaying sites, and the `train` task name that the clients support. The two required components ids are also mapped to the corresponding `PTFileModelPersistor` and `FullModelShareableGenerator` defined under components.

Let's use the Job CLI to create the job from a Cyclic template:

In [ ]:
! nvflare config -jt ../../../../../job_templates

In [ ]:
! nvflare job create -j /tmp/nvflare/jobs/cyclic_pt -w cyclic_pt -sd ../code/fl -force

We can take a look at the server and client configurations and make any changes as desired:

In [ ]:
! cat /tmp/nvflare/jobs/cyclic_pt/app/config/config_fed_server.conf

In [ ]:
! cat /tmp/nvflare/jobs/cyclic_pt/app/config/config_fed_client.conf

Ensure that the `app_script` is set to the Client API FL `train.py` code and the model path for the persistor is set to the `net.Net`.

## Prepare Data

Make sure the CIFAR10 dataset is downloaded with the following script:

In [ ]:
! python ../data/download.py

## Run the Job

Now we can run the job with the simulator:

In [ ]:
! nvflare simulator /tmp/nvflare/jobs/cyclic_pt -w /tmp/nvflare/cyclic_pt_workspace -t 2 -n 2 

As an additional resource, also see the [hello-cyclic](../../../../hello-world/hello-cyclic/README.md) for a Tensorflow Executor implementation using the MNIST dataset.

While this example focused on the server-controlled cyclic workflow, now we will introduce the idea of client-controlled workflows.
The next [cyclic_ccwf](../cyclic_ccwf/cyclic_ccwf.ipynb) example is a client-controlled version of the cyclic workflow.